<a href="https://colab.research.google.com/github/elviobg/titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

import os
ROOT_PATH = '/content/drive/My Drive/Colab Notebooks/Titanic/'
DATASETS_PATH = '/content/drive/My Drive/Colab Notebooks/Titanic/dataset/'
os.chdir(ROOT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np

In [12]:
train_original = pd.read_csv(DATASETS_PATH+'train.csv')
test_original = pd.read_csv(DATASETS_PATH+'test.csv')
print("Data loaded...")
train_original.head()

Data loaded...


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
train_original = train_original.rename(columns = {'Survived': 'survived', 
                                                  'Pclass': 'pclass', 
                                                  'Name': 'name', 
                                                  'Sex': 'sex', 
                                                  'Age': 'age', 
                                                  'SibSp': 'sibsp',
                                                  'Parch': 'parch', 
                                                  'Ticket': 'ticket',
                                                  'Fare': 'fare', 
                                                  'Embarked': 'embarked'}, inplace = False)

test_original = test_original.rename(columns = {'Survived': 'survived', 
                                                  'Pclass': 'pclass', 
                                                  'Name': 'name', 
                                                  'Sex': 'sex', 
                                                  'Age': 'age', 
                                                  'SibSp': 'sibsp',
                                                  'Parch': 'parch', 
                                                  'Ticket': 'ticket',
                                                  'Fare': 'fare',                                                   
                                                  'Embarked': 'embarked'}, inplace = False)

train_original = train_original.drop(['PassengerId','Cabin'], axis=1)
test_original = test_original.drop(['Cabin'], axis=1)

In [15]:
train_original.info()
test_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   name      891 non-null    object 
 3   sex       891 non-null    object 
 4   age       714 non-null    float64
 5   sibsp     891 non-null    int64  
 6   parch     891 non-null    int64  
 7   ticket    891 non-null    object 
 8   fare      891 non-null    float64
 9   embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   pclass       418 non-null    int64  
 2   name         418 non-null    object 
 3   sex          418 non-null    object 
 4   ag

In [17]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
def organize_titles(dataset):
  dataset['title'] = dataset.name.str.extract(' ([A-Za-z]+)\.', expand=False)
  dataset['title'] = dataset['title'].map(title_mapping)
  dataset['title'] = dataset['title'].fillna(0)

organize_titles(train_original)
organize_titles(test_original)
train_original = train_original.drop(['name'], axis=1)
test_original = test_original.drop(['name'], axis=1)
train_original.head()

,survived,pclass,sex,age,sibsp,parch,ticket,fare,embarked,title
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S,1.0
1,1,1,female,38.0,1,0,PC 17599,71.2833,C,3.0
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,2.0
3,1,1,female,35.0,1,0,113803,53.1000,S,3.0
4,0,3,male,35.0,0,0,373450,8.0500,S,1.0


In [18]:
train_original['sex'] = train_original['sex'].map( {'female': 1, 'male': 0} ).astype(int)
test_original['sex'] = test_original['sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [19]:
def age_fill(dataset):
  guess_ages = np.zeros((2,3))
  for i in range(0, 2):
    for j in range(0, 3):
      guess_df = dataset[(dataset['sex'] == i) & (dataset['pclass'] == j+1)]['age'].dropna()
      age_guess = guess_df.median()
      guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
        
  for i in range(0, 2):
    for j in range(0, 3):
      dataset.loc[ (dataset.age.isnull()) & (dataset.sex == i) & (dataset.pclass == j+1),'age'] = guess_ages[i,j]
  dataset['age'] = dataset['age'].astype(int)

def age_organize(dataset):
  dataset.loc[ dataset['age'] <= 16, 'age'] = 0
  dataset.loc[(dataset['age'] > 16) & (dataset['age'] <= 32), 'age'] = 1
  dataset.loc[(dataset['age'] > 32) & (dataset['age'] <= 48), 'age'] = 2
  dataset.loc[(dataset['age'] > 48) & (dataset['age'] <= 64), 'age'] = 3
  dataset.loc[ dataset['age'] > 64, 'age'] = 4
  dataset['age*class'] = dataset.age * dataset.pclass

age_fill(train_original)
age_organize(train_original)

age_fill(test_original)
age_organize(test_original)

train_original.info()
test_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   survived   891 non-null    int64  
 1   pclass     891 non-null    int64  
 2   sex        891 non-null    int64  
 3   age        891 non-null    int64  
 4   sibsp      891 non-null    int64  
 5   parch      891 non-null    int64  
 6   ticket     891 non-null    object 
 7   fare       891 non-null    float64
 8   embarked   889 non-null    object 
 9   title      891 non-null    float64
 10  age*class  891 non-null    int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 76.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   pclass       418 non-null    int64  
 2   sex          418 non-null    int64

In [21]:
train_original['familysize'] = train_original['sibsp'] + train_original['parch'] + 1
test_original['familysize'] = test_original['sibsp'] + test_original['parch'] + 1

def familly(dataset):
    dataset['isalone'] = 0
    dataset.loc[dataset['familysize'] == 1, 'isalone'] = 1

familly(train_original)
familly(test_original)
train_original = train_original.drop(['sibsp', 'parch', 'familysize'], axis=1)
test_original = test_original.drop(['sibsp', 'parch', 'familysize'], axis=1)
train_original.head()

,survived,pclass,sex,age,sibsp,parch,ticket,fare,embarked,title,age*class,familysize,isalone
0,0,3,0,1,1,0,A/5 21171,7.2500,S,1.0,3,2,0
1,1,1,1,2,1,0,PC 17599,71.2833,C,3.0,2,2,0
2,1,3,1,1,0,0,STON/O2. 3101282,7.9250,S,2.0,3,1,1
3,1,1,1,2,1,0,113803,53.1000,S,3.0,2,2,0
4,0,3,0,2,0,0,373450,8.0500,S,1.0,6,1,1


In [22]:
def embark(dataset, freq_port):
  dataset['embarked'] = dataset['embarked'].fillna(freq_port)
  dataset['embarked'] = dataset['embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

freq_port = train_original.embarked.dropna().mode()[0]
embark(train_original, freq_port)
embark(test_original, freq_port)
train_original.head()

,survived,pclass,sex,age,sibsp,parch,ticket,fare,embarked,title,age*class,familysize,isalone
0,0,3,0,1,1,0,A/5 21171,7.2500,0,1.0,3,2,0
1,1,1,1,2,1,0,PC 17599,71.2833,1,3.0,2,2,0
2,1,3,1,1,0,0,STON/O2. 3101282,7.9250,0,2.0,3,1,1
3,1,1,1,2,1,0,113803,53.1000,0,3.0,2,2,0
4,0,3,0,2,0,0,373450,8.0500,0,1.0,6,1,1


In [23]:
def fare_organize(dataset):
  dataset.loc[ dataset['fare'] <= 7.91, 'fare'] = 0
  dataset.loc[(dataset['fare'] > 7.91) & (dataset['fare'] <= 14.454), 'fare'] = 1
  dataset.loc[(dataset['fare'] > 14.454) & (dataset['fare'] <= 31), 'fare']   = 2
  dataset.loc[ dataset['fare'] > 31, 'fare'] = 3
 

train_original['fare'].fillna(train_original['fare'].dropna().median(), inplace=True)
test_original['fare'].fillna(test_original['fare'].dropna().median(), inplace=True)
fare_organize(train_original)
fare_organize(test_original)
train_original.head()

,fare,survived
0,0.0,0.197309
1,1.0,0.308756
2,2.0,0.445415
3,3.0,0.581081


In [24]:
train_original = train_original.drop(['ticket'], axis=1)
test_original = test_original.drop(['ticket'], axis=1)
train_original.head()

,survived,pclass,sex,age,fare,embarked,title,age*class,isalone
0,0,3,0,1,0.0,0,1.0,3,0
1,1,1,1,2,3.0,1,3.0,2,0
2,1,3,1,1,1.0,0,2.0,3,1
3,1,1,1,2,3.0,0,3.0,2,0
4,0,3,0,2,1.0,0,1.0,6,1
